In [1]:
from sklearn.cross_validation import KFold
import pandas as pd
import numpy as np

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('../data/processed/cat_week_time_264.csv')
data = data.fillna(0)
print data.shape
data.head()

(2101, 267)


,census_tract,label,label_bucket,num_checkins,num_park,num_fri,num_4pm_8pm,num_thu,num_12pm_4pm,num_bus_station,...,num_swiss_restaurant,num_peruvian_restaurant,num_gluten_free_restaurant,num_funeral_home,num_law_school,num_pet_service,num_recycling_facility,num_internet_cafe,num_motorcycle_shop,num_planetarium
0,36005000200,72034.0,54596 <= x <= 73345,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36005000400,74836.0,73346 <= x <= 244375,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36005001600,32312.0,9829 <= x <= 39457,76.0,0.0,41.0,16.0,1.0,17.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36005001900,37936.0,9829 <= x <= 39457,77.0,0.0,1.0,33.0,5.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,36005002000,18086.0,9829 <= x <= 39457,4.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X = data.as_matrix()[:, 3:len(data)-1].astype(int) # take all cols after first 2
y = data['label_bucket'].as_matrix()

from sklearn import preprocessing
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
X_norm = min_max_scaler.fit_transform(X)

/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [4]:
data.head(1)

,census_tract,label,label_bucket,num_checkins,num_park,num_fri,num_4pm_8pm,num_thu,num_12pm_4pm,num_bus_station,...,num_swiss_restaurant,num_peruvian_restaurant,num_gluten_free_restaurant,num_funeral_home,num_law_school,num_pet_service,num_recycling_facility,num_internet_cafe,num_motorcycle_shop,num_planetarium
0,36005000200,72034.0,54596 <= x <= 73345,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from sklearn.model_selection import train_test_split

NUM_FOLDS = 10
test_size = 1.0/(NUM_FOLDS)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=test_size)

### sort the features by weights of a logistic regression model

In [6]:
from sklearn import linear_model

(M, n) = X_train.shape
# kf = KFold(M, n_folds=2) # Define the split (5 folds)kf:

train_accuracies = []
test_accuracies = []
weights = {}
num_iters = 0

print '**fold**'

# add l1 penalty for sparse logistic regression
lr = linear_model.LogisticRegression(penalty='l1')

clf = lr.fit(X_train, y_train)
weights = clf.coef_

y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

train_acc = 1.0*(y_train == y_train_pred).sum()/ len(y_train)
train_accuracies.append(train_acc)
print 'train acc:', train_acc

test_acc = 1.0*(y_test == y_test_pred).sum()/ len(y_test)
test_accuracies.append(test_acc)
print 'test acc:', test_acc
print '---\n'

(weights_M, weights_N) = weights.shape

for i in range(weights_M):
    max_weights = np.argpartition(weights[i], -20)[-20:]
    best_predictors = []
    best_weights = []
    for mw in max_weights:
        best_predictors.append(data.columns[mw+3])
        best_weights.append(mw)
    print 'Best predictors for Bucket #', i+1, ': ', ', '.join(best_predictors)
    print '---\n'
print '---\n'

print 'train mean:', np.mean(train_accuracies)
print 'test mean:', np.mean(test_accuracies)

**fold**
train acc: 0.370899470899
test acc: 0.331753554502
---

Best predictors for Bucket # 1 :  num_burger_joint, num_paper_office_supplies_store, num_music_venue, num_light_rail, num_automotive_shop, num_south_american_restaurant, num_fast_food_restaurant, num_ski_area, num_stadium, num_medical_center, num_sandwich_place, num_thrift_vintage_store, num_theater, num_university, num_cosmetics_shop, num_plaza, num_snack_place, num_laundry_service, num_subway, num_bike_rental_bike_share
---

Best predictors for Bucket # 2 :  num_laundry_service, num_bridal_shop, num_college_stadium, num_ski_area, num_automotive_shop, num_diner, num_stadium, num_medical_center, num_sandwich_place, num_middle_school, num_arepa_restaurant, num_thrift_vintage_store, num_theater, num_spanish_restaurant, num_university, num_cosmetics_shop, num_cuban_restaurant, num_tattoo_parlor, num_plaza, num_greek_restaurant
---

Best predictors for Bucket # 3 :  num_other_great_outdoors, num_art_museum, num_design_studio,